# MobileNet

In [4]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.models import load_model
import keras

import numpy as np
from sklearn.metrics import confusion_matrix,classification_report
import matplotlib.pyplot as plt

In [44]:
#--------------#
# custom metrics
#--------------#
# for custom metrics
import keras.backend as K
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet import mobilenet
from keras.layers import Input
# load MobileNet
img_rows, img_cols = 224, 224 
img_width, img_height = 224,224
weights='weights/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5'




# fix data format issues
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
# Re-loads the MobileNet model without the top or FC layers

mobile = keras.applications.mobilenet.MobileNet(input_shape=input_shape,
                                                   alpha=1.0, 
                                                   depth_multiplier=1, 
                                                   dropout=1e-3, 
                                                   include_top=False, 
                                                   weights='imagenet')



# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in mobile.layers:
    layer.trainable = False

In [15]:
import pandas as pd
pd.set_option('max_colwidth', -1)
layers = [(layer, layer.name, layer.trainable) for layer in mobile.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']) 

,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at 0x000001904030A1D0>,input_4,False
1,<keras.layers.convolutional.ZeroPadding2D object at 0x00000190402F44E0>,conv1_pad,False
2,<keras.layers.convolutional.Conv2D object at 0x00000190402F45F8>,conv1,False
3,<keras.layers.normalization.BatchNormalization object at 0x000001904028B5F8>,conv1_bn,False
4,<keras.layers.advanced_activations.ReLU object at 0x00000190402B44E0>,conv1_relu,False
5,<keras.layers.convolutional.DepthwiseConv2D object at 0x000001903F54DAC8>,conv_dw_1,False
6,<keras.layers.normalization.BatchNormalization object at 0x00000190404C5E80>,conv_dw_1_bn,False
7,<keras.layers.advanced_activations.ReLU object at 0x000001903F570F98>,conv_dw_1_relu,False
8,<keras.layers.convolutional.Conv2D object at 0x000001903F616160>,conv_pw_1,False
9,<keras.layers.normalization.BatchNormalization object at 0x000001903F62B898>,conv_pw_1_bn,False


In [41]:
base_model = applications.InceptionV3(weights='imagenet', 
                                      include_top=False, # remove top fully connected layers
                                      input_shape=(img_width, img_height, 3))
# compile model using Adam optimizer, categorical cross entropy loss
# use low learning rate for transfer learning
model.compile(optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, 
                             epsilon=1e-08, decay=0.0),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
# build a classifier model to put on top of the convolutional model
model_top = Sequential()
model_top.add(GlobalAveragePooling2D(
    input_shape=base_model.output_shape[1:],
    data_format=None)),
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(4, activation='softmax'))
model = Model(inputs=base_model.input, 
              outputs=model_top(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
#top_model.load_weights(top_model_weights_path)



In [47]:
# define directories and parameters
train_data_dir = 'data/train'
test_data_dir = 'data/test'
validation_data_dir = 'data/validation'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)

In [48]:
# generator/ load data
batch_size=16
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 83484 images belonging to 4 classes.
Found 968 images belonging to 4 classes.


In [49]:
#--------------#
# train top layers
#--------------#
checkpoint = ModelCheckpoint("retinal_mobile1.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

callbacks = [earlystop, checkpoint]

In [50]:
mobile.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr = 0.001),
              metrics = ['accuracy'])

In [51]:
nb_train_samples = 83484
nb_validation_samples = 968
model = mobile
epochs = 10
batch_size = 16

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

In [ ]:
scores = model.evaluate_generator(validation_generator,steps=nb_validation_samples // batch_size+1, verbose=1)
print('\nTest result: %.3f loss: %.3f' %(scores[1]*100,scores[0]))

In [ ]:
model.save("retinal_mobile.h5")

In [ ]:
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

In [ ]:
class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())

In [ ]:
y_pred = model.predict_generator(validation_generator, nb_validation_samples // batch_size+1)

In [ ]:
y_pred_label = np.argmax(y_pred, axis=1)

In [ ]:
validation_generator.classes.shape,y_pred_label.shape

In [ ]:
# Confusion Matrix and Classification Report
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred_label))
print('Classification Report')
print(classification_report(validation_generator.classes, y_pred_label, target_names=classes))

In [ ]:
plt.figure(figsize=(8,8))
cnf_matrix = confusion_matrix(validation_generator.classes, y_pred_label)

plt.imshow(cnf_matrix, interpolation='nearest')
plt.colorbar()
tick_marks = np.arange(len(classes))
_ = plt.xticks(tick_marks, classes, rotation=90)
_ = plt.yticks(tick_marks, classes)
plt.show();

In [ ]:
#--------------#
# plot acc and loss two plot
#--------------#
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# load classifier
from keras.applications import MobileNet
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D,GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.models import load_model

import numpy as np
from sklearn.metrics import confusion_matrix,classification_report
import matplotlib.pyplot as plt

classifier = load_model("retinal_mobile.h5")